# Search the AuScope Portal Data for CSW Records
## An interactive demonstration of accessing the AuScope Portal using the auscopecat Python library available from Pypi.
### https://pypi.org/project/auscopecat/

In [ ]:
! pip install -i https://test.pypi.org/simple/ auscopecat
! pip install ipywidgets
! pip install ipyleaflet
# If the above ipyleaflet pip installation does not work, you may need to try conda
#! conda install -y -c conda-forge ipyleaflet

In [1]:
import ipywidgets as widgets
from ipyleaflet import GeomanDrawControl, Map, basemaps
from IPython.display import Markdown, display

from auscopecat import api
from auscopecat.auscopecat_types import SpatialSearchType

AUSTRALIA_BBOX = {
    'north': -10.6681, 'east': 153.5694,
    'south': -43.6345, 'west': 113.3389
}

In [ ]:
display(Markdown('## Select Bounds'))
display(Markdown('### Enter values or draw box on the right'))

australia_btn = widgets.Button(
    description = 'Select Australia',
    button_style = '',
    tooltip = 'Reset bounds to Australia',
    icon = ''
)
display(australia_btn)

north_field = widgets.BoundedFloatText(
    description = 'North',
    value = AUSTRALIA_BBOX['north'],
    min = -90.0,
    max = 90.0,
    style = {
        'font_size': '170%'
    }
)

west_field = widgets.BoundedFloatText(
    description = 'West',
    value = AUSTRALIA_BBOX['west'],
    min = -180.0,
    max = 180.0,
    style = {
        'font_size': '170%'
    }
)

south_field = widgets.BoundedFloatText(
    description = 'South',
    value = AUSTRALIA_BBOX['south'],
    min = -90.0,
    max = 90.0,
    style = {
        'font_size': '170%'
    }
)

east_field = widgets.BoundedFloatText(
    description = 'East',
    value = AUSTRALIA_BBOX['east'],
    min = -180.0,
    max = 180.0,
    style = {
        'font_size': '170%'
    }
)
display(Markdown('\n#### Bounding Box'))
display(north_field)
display(west_field)
display(south_field)
display(east_field)

def reset_bounds(b):
    north_field.value = AUSTRALIA_BBOX['north']
    south_field.value = AUSTRALIA_BBOX['south']
    west_field.value = AUSTRALIA_BBOX['west']
    east_field.value = AUSTRALIA_BBOX['east']

australia_btn.on_click(reset_bounds)

m = Map(basemap=basemaps.Esri.WorldStreetMap, center = (-29.6, 133.0), zoom = 5)

draw_control = GeomanDrawControl()
draw_control.circlemarker = {}
draw_control.polyline = {}
draw_control.polygon = {}
draw_control.rectangle = {
    'shapeOptions': {
        'fillColor': '#fca45d',
        'color': '#fca45d',
        'fillOpacity': 0.7
    }
}

def handle_draw(target, action, geo_json):
    if action == 'create' or action == 'drag':
        coords = geo_json[0]['geometry']['coordinates'][0]
        if len(coords) == 5:
            west_field.value = str(coords[0][0])
            south_field.value = str(coords[0][1])
            east_field.value = str(coords[2][0])
            north_field.value = str(coords[2][1])
            # TODO: Enable Rectangle control (possible?)
    elif action == 'remove':
        # TODO: Disable Rectangle control (possible?)
        pass


draw_control.on_draw(handle_draw)
m.add(draw_control)
display(m)

In [ ]:
search_field = widgets.Text(
    continuous_update = False,
    placeholder = 'Enter query and press Enter or click Search button',
    style = {
        'font_size': '170%'
    }
)
search_field.placeholder = 'Enter search term...'
search_results_output = widgets.Output()

search_btn = widgets.Button(
    description = 'Search',
    icon = 'search',
    tooltip = 'Search'
)

clear_search_btn = widgets.Button(
    description = 'Clear',
    tooltip = 'Clear search',
    icon = 'times'
)

search_box_layout = widgets.Layout(
    display = 'flex',
    flex_flow = 'row',
    align_items = 'stretch'
)
search_box = widgets.Box(children=[search_field, search_btn, clear_search_btn], layout = search_box_layout)
display(Markdown('## Search For Results'))
display(search_box)
display(search_results_output)

bbox = {}

def search(query):
    search_results_output.clear_output()
    with search_results_output:
        try:
            bbox['north'] = float(north_field.value)
            bbox['west'] = float(west_field.value)
            bbox['south'] = float(south_field.value)
            bbox['east'] = float(east_field.value)
        except Exception as e:
            display(Markdown(f'## Error with bounds: {e}'))
            return

        try:
            search_results = api.search_records(query, spatial_search_type=SpatialSearchType.INTERSECTS, bbox=bbox)
        except Exception as e:
            display(Markdown(f'## Error searching records: {e}'))
            return
        if len(search_results) > 0:
            display(Markdown(f'\n## Results ({len(search_results)}):\n'))
            display(search_results)
        else:
            display(Markdown('## No results were found'))

def on_search_output(s):
    search(s.new)

def on_search_clicked(b):
    search(search_field.value)

def on_clear_search_clicked(b):
    search_field.value = ''
    search_results_output.clear_output()

search_field.observe(on_search_output, 'value')
clear_search_btn.on_click(on_clear_search_clicked)
